In [30]:
import os
import random
import numpy as np
import pandas as pd
from collections import defaultdict
from tsfresh import extract_features
import torch
import seaborn as sns
import matplotlib.pyplot as plt
from torch import nn
from torch.autograd import Variable
import gc
import lightgbm as lgb
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, log_loss, confusion_matrix, accuracy_score, roc_auc_score, roc_curve
%pwd
%cd /Users/wenxindong/Desktop/Stanford/CS329P/project/riiid-test-answer-prediction


/Users/wenxindong/Desktop/Stanford/CS329P/project/riiid-test-answer-prediction


## load files

In [31]:
train_pickle = 'train_39360_users_preprocessed.pickle'  #about one tenth of the training dataset
valid_pickle = 'valid_4920_users_preprocessed.pickle'  
test_pickle = "test_4920_users_preprocessed.pickle"
question_file = 'questions.csv'

# Read data
train = pd.read_pickle(train_pickle)
valid = pd.read_pickle(valid_pickle)
test = pd.read_pickle(test_pickle)

questions_df = pd.read_csv(question_file)
train = train.fillna(0)
valid = valid.fillna(0)
test = test.fillna(0)

#subsample
train = train[:len(train)//10]
valid = valid[:len(valid)//10]
test = test[:len(test)//10]

for i in range(1,8):
  train['part'+str(i)] = (train["part"]==i)
  valid['part'+str(i)] = (valid["part"]==i)
  test['part'+str(i)] = (test["part"]==i)

#get labels
TARGET = 'answered_correctly'
train_x = train.drop([TARGET], axis = 1)
valid_x = valid.drop([TARGET], axis = 1)
test_x = test.drop([TARGET], axis = 1)

train_y = train[TARGET]
valid_y = valid[TARGET]
test_y = test[TARGET]

#start with 4 features only
column_features = ["explanation_u_avg", "timestamp_u_incorrect_recency", "explanation_q_avg", "prior_question_had_explanation","part", "timestamp_u_recency_3", "timestamp_u_recency_2", "timestamp_u_recency_1","prior_question_elapsed_time", "answered_correctly_u_avg","elapsed_time_u_avg",  "answered_correctly_uq_count", "elapsed_time_q_avg", "answered_correctly_q_avg"]
train_x = train_x.drop([ "timestamp_u_incorrect_recency", "explanation_q_avg", "prior_question_had_explanation","part", "timestamp_u_recency_3", "timestamp_u_recency_2", "timestamp_u_recency_1", "prior_question_elapsed_time"], axis= 1)
valid_x = valid_x.drop([ "timestamp_u_incorrect_recency", "explanation_q_avg", "prior_question_had_explanation","part", "timestamp_u_recency_3", "timestamp_u_recency_2", "timestamp_u_recency_1", "prior_question_elapsed_time"], axis=1)
test_x = test_x.drop([ "timestamp_u_incorrect_recency", "explanation_q_avg", "prior_question_had_explanation","part", "timestamp_u_recency_3", "timestamp_u_recency_2", "timestamp_u_recency_1", "prior_question_elapsed_time"], axis=1)

#binary classification acc = 0.97
mean = train_x.mean()
std = train_x.std()
train_x = (train_x -train_x.mean())/train_x.std()
valid_x = (valid_x -mean)/std
test_x = (test_x -mean)/std


print(train_x.shape, train_y.shape)
print(valid_x.shape, valid_y.shape)
print(test_x.shape, test_y.shape)

train_x.describe()


(989711, 13) (989711,)
(126725, 13) (126725,)
(119116, 13) (119116,)


,answered_correctly_u_avg,elapsed_time_u_avg,explanation_u_avg,answered_correctly_q_avg,elapsed_time_q_avg,answered_correctly_uq_count,part1,part2,part3,part4,part5,part6,part7
count,9.897110e+05,9.897110e+05,9.897110e+05,9.897110e+05,9.897110e+05,9.897110e+05,9.897110e+05,9.897110e+05,9.897110e+05,9.897110e+05,9.897110e+05,9.897110e+05,9.897110e+05
mean,9.243005e-07,1.228034e-07,-3.100500e-07,-8.752863e-07,-6.015538e-07,5.226527e-17,-1.823541e-17,-2.791310e-17,-3.348423e-17,6.892123e-18,8.327982e-17,-6.777255e-18,-9.786815e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-4.711462e+00,-3.533400e+00,-2.822415e+00,-3.068889e+00,-2.834124e+00,-3.307495e-01,-2.891807e-01,-4.848797e-01,-3.121246e-01,-3.029209e-01,-8.169098e-01,-3.515809e-01,-2.279563e-01
25%,-4.652698e-01,-6.408426e-01,7.916185e-02,-5.927076e-01,-5.646055e-01,-3.307495e-01,-2.891807e-01,-4.848797e-01,-3.121246e-01,-3.029209e-01,-8.169098e-01,-3.515809e-01,-2.279563e-01
50%,1.432532e-01,-1.490851e-01,4.294540e-01,1.103226e-01,-1.592306e-01,-3.307495e-01,-2.891807e-01,-4.848797e-01,-3.121246e-01,-3.029209e-01,-8.169098e-01,-3.515809e-01,-2.279563e-01
75%,6.715313e-01,4.410607e-01,5.616751e-01,7.290017e-01,2.662202e-01,-3.307495e-01,-2.891807e-01,-4.848797e-01,-3.121246e-01,-3.029209e-01,1.224124e+00,-3.515809e-01,-2.279563e-01
max,2.483475e+00,3.789045e+01,6.359219e-01,1.592159e+00,3.072918e+01,2.323279e+01,3.458042e+00,2.062365e+00,3.203846e+00,3.301188e+00,1.224124e+00,2.844293e+00,4.386801e+00


In [59]:
#soft labels 
autogl_predict_proba = pd.read_pickle("autogl_soft_label.pickle")
autogl_predict_proba = np.array(autogl_predict_proba[[1]])
autogl_predict_proba.shape

(989711, 1)

## Soft decision tree

In [68]:
class SoftDecisionTreeNode(nn.Module):
    def __init__(self, in_features=10):
        super().__init__()
        self.linear = torch.nn.Linear(in_features, 1, bias = True)
    def forward(self,x):
        temp = self.linear(x)
        output = nn.Sigmoid()(temp)
        return output

class SoftDecisionTreeLevel(nn.Module):
    def __init__(self, in_features=10, depth = 1, weight = None, bias = None):
        super().__init__()
        
        self.num_nodes = 2**(depth-1)
        self.in_features = in_features
        self.weight = weight
        if self.weight !=None:
            self.weight = weight
            self.bias = bias
        else:
            self.linear = torch.nn.Linear(in_features, self.num_nodes, bias = True)
            
    def forward(self,x):
        temp = None
        if self.weight !=None:
            temp = x@ self.weight.T + self.bias
        else:
            temp = self.linear(x)
        output = nn.Sigmoid()(temp)
        return output
    
class SoftDecisionTreeSparseLevel(nn.Module):
    #one feature per node 
    #option 1: specify feature per node (to determine feature order)
    #option 2: let feature be i for level i (baseline, same featureo on each level)
    def __init__(self, in_features=10, freeze=False, depth = 1, feature = 0):
        
        #feature could be an int or arr
        super().__init__()
        self.num_nodes = 2**(depth-1)
        self.in_features = in_features
        self.feature = feature
        self.freeze = freeze
        if self.freeze: #option 1
            for i in range(self.num_nodes):
                setattr(self, "linear" + str(i), SoftDecisionTreeNode(in_features= 1))
        else:
            self.linear = torch.nn.Linear(1, self.num_nodes, bias = True) 
    def forward(self,x):
        temp = torch.zeros((x.shape[0], self.num_nodes))
        
        if self.freeze:
            for i in range(self.num_nodes):
                node = getattr(self, "linear"+str(i))(x[:, [self.feature[i]]])
                temp[:, [i]] = node               
        else:
            temp = self.linear(x[:, [self.feature]])
        output = nn.Sigmoid()(temp)
        return output
    
class SoftDecisionTree(nn.Module):
    def __init__(self, in_features=10,depth=2, sparse = True, weights = None, biases = None, freeze= False, features = None):
        super().__init__()
        self.in_features = in_features
        self.depth = depth
        self.freeze = freeze
        self.sparse = sparse
        for d in range(1, self.depth+1):
            if self.sparse:
                f = d-1
                if self.freeze:
                    f = features[d-1]
                setattr(self, "level" + str(d), SoftDecisionTreeSparseLevel(in_features= self.in_features, depth = d, feature = f, freeze=freeze))
            else:
                if weights ==None:
                    setattr(self, "level" + str(d), SoftDecisionTreeLevel(in_features= self.in_features, depth = d))
                else:
                    setattr(self, "level" + str(d), SoftDecisionTreeLevel(in_features= self.in_features, depth = d, weight = weights[d-1], bias = biases[d-1]))

    def forward(self, x):
        probs = []
        for d in range(1, self.depth+1):
            probs.append(getattr(self, "level"+str(d))(x))
        temp = probs[-1]
        for i in range(self.depth-2, -1, -1):
            temp = probs[i]*(temp.reshape(-1,2)[:,1].reshape(-1, 2**i)) + (1-probs[i])*(temp.reshape(-1,2)[:,0].reshape(-1, 2**i))
        prediction = temp
        return prediction

#dummy dataset
# x_train = np.array([[0,1],[1,1],[2,2],[3,3],[4,4],[5,5],[6,6],[7,7], [8,6], [9,9]], dtype=np.float32)
# y_train = np.array([0,0,1,1,0,0,0,0, 1, 1], dtype=np.float32).reshape(-1, 1)
# y_train = y_train.reshape(-1, 1)


#riiid dataset
x_train = train_x.to_numpy(dtype=np.float32)
x_valid = valid_x.to_numpy(dtype=np.float32)
x_test = test_x.to_numpy(dtype=np.float32)

y_train = train_y.to_numpy(dtype=np.float32).reshape(-1, 1)
y_valid = valid_y.to_numpy(dtype=np.float32).reshape(-1, 1)
y_test = test_y.to_numpy(dtype=np.float32).reshape(-1, 1)

print("training features", x_train.shape)
print("training labels", y_train.shape)

def evaluate(model, visualize = False, inputDim = 13):
    with torch.no_grad():
        if torch.cuda.is_available():
            predicted = model(Variable(torch.from_numpy(x_train).cuda())).cpu().data.numpy()
        else:
            predicted_train = model(Variable(torch.from_numpy(x_train))).data.numpy()
            predicted_valid = model(Variable(torch.from_numpy(x_valid))).data.numpy()
            predicted_test = model(Variable(torch.from_numpy(x_test))).data.numpy()
            
            final_prediction_train = predicted_train>0.5
            final_prediction_valid = predicted_valid>0.5
            final_prediction_test = predicted_test>0.5
            accuracy_train = accuracy_score(y_train, final_prediction_train)
            accuracy_valid = accuracy_score(y_valid, final_prediction_valid)
            accuracy_test = accuracy_score(y_test, final_prediction_test)
            
            auc_valid = roc_auc_score(y_valid, predicted_valid)
            auc_test = roc_auc_score(y_test, predicted_test)
            
            print("training acc {}, validation acc {} test acc {}".format(accuracy_train,accuracy_valid, accuracy_test))
            print("validation auc {} test auc {}".format(auc_valid, auc_test))
    if visualize:
        for i in range(inputDim):
            color = ['red' if x==1 else 'black' for x in y_train]
            plt.scatter(x_valid[:, i], predicted_valid, c= color, alpha=0.5)
            plt.legend(loc='best')
            plt.title("feature"+str(i))
            plt.show()


def train_softtree(depth = 4, sparse=True, epochs = 100, weights = None, 
                   biases = None, visualize = True, verbose=False, freeze=False, features = None, lr=0.01, soft_labels = False):
    
    #model
    inputDim = x_train.shape[1]       
    outputDim = 1       
    learningRate = lr
    model = SoftDecisionTree(in_features = inputDim, depth=depth, sparse=sparse, freeze = freeze,
                             weights = weights, biases = biases,features = features)
#     if verbose: print(model)
    print("Soft: sparse= {}, {} epochs, {} input dimension, {} learing rate, {} depth".format(sparse, epochs, inputDim, learningRate, depth))

    #train model
    if torch.cuda.is_available():
        model.cuda()
    criterion = torch.nn.BCELoss() 
    optimizer = torch.optim.Adam(model.parameters(), lr=learningRate)
    train_loss =[]
    valid_loss = []
    if torch.cuda.is_available():
        inputs = Variable(torch.from_numpy(x_train).cuda())
        if soft_labels:
            labels = Variable(torch.from_numpy(autogl_predict_proba).cuda())
        else:
            labels = Variable(torch.from_numpy(y_train).cuda())
        inputs_val = Variable(torch.from_numpy(x_valid).cuda())
        labels_val = Variable(torch.from_numpy(y_valid).cuda())
    else:
        inputs = Variable(torch.from_numpy(x_train))
        if soft_labels:
            labels = Variable(torch.from_numpy(autogl_predict_proba))
        else:
            labels = Variable(torch.from_numpy(y_train))
        inputs_val = Variable(torch.from_numpy(x_valid))
        labels_val = Variable(torch.from_numpy(y_valid))
    print("EPOCH 0")
    evaluate(model)
    
    for epoch in range(epochs):

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = - torch.sum((torch.log(outputs)*labels + torch.log(1-outputs)*(1-labels))) / outputs.shape[0]
        #criterion(outputs, labels)
        
        l1_lambda = 0.001
        reg = l1_lambda * sum(p.abs().sum() for name, p in model.named_parameters() if "weight" in name)
        loss.backward()
        if not sparse:
            reg.backward()
        optimizer.step()
        if epoch%10==0:
            
            with torch.no_grad():
                outputs_valid = model(inputs_val)
                loss_valid = criterion(outputs_valid, labels_val)
                valid_loss.append(loss_valid)
                train_loss.append(loss)
            if verbose: 
                print("EPOCH {}".format(epoch))
                evaluate(model, visualize = visualize, inputDim = inputDim)
    evaluate(model, visualize = visualize, inputDim = inputDim)
        #inference
    if visualize:
        plt.plot(valid_loss, label = "validation loss")
        plt.plot(train_loss, label = "training loss")
        plt.legend(loc='best')
        plt.show()

    return model
#compare model performance 

# #normal decision tree
# NormalDecisionTree(max_depth=1)
# NormalDecisionTree(max_depth=2)
# NormalDecisionTree(max_depth=3)
# NormalDecisionTree(max_depth=4)
# NormalDecisionTree(max_depth=5)
# NormalDecisionTree(max_depth=6)
# print()
# #soft decision tree with sparse nodes:
# train_softtree(depth = 1, sparse=True, epochs = 100, visualize = False, verbose=False)
# train_softtree(depth = 2, sparse=True, epochs = 100, visualize = False, verbose=False)
# train_softtree(depth = 3, sparse=True, epochs = 100, visualize = False, verbose=False)
# train_softtree(depth = 4, sparse=True, epochs = 100, visualize = False, verbose=False)
# train_softtree(depth = 5, sparse=True, epochs = 100, visualize = False, verbose=False)
# deterministic_sparse_model = train_softtree(depth = 6, sparse=True, epochs = 100, visualize = False, verbose=False)
# print()

# #soft decision tree with non-sparse nodes:
# train_softtree(depth = 1, sparse=False, epochs = 100, visualize = False, verbose=False)
# train_softtree(depth = 2, sparse=False, epochs = 100, visualize = False, verbose=False)
# train_softtree(depth = 3, sparse=False, epochs = 100, visualize = False, verbose=False)
# train_softtree(depth = 4, sparse=False, epochs = 100, visualize = False, verbose=False)
# train_softtree(depth = 5, sparse=False, epochs = 100, visualize = False, verbose=False)

        
train_softtree(depth = 4, sparse=False, epochs = 150, visualize = False, verbose=True)


training features (989711, 13)
training labels (989711, 1)
Soft: sparse= False, 150 epochs, 13 input dimension, 0.01 learing rate, 4 depth
EPOCH 0
training acc 0.4785952666990667, validation acc 0.4802682974945749 test acc 0.48128714866180866
validation auc 0.46935931146228915 test auc 0.46674931254340996
EPOCH 0
training acc 0.49321367550729456, validation acc 0.49235746695600713 test acc 0.4979012055475335
validation auc 0.4973710002532302 test auc 0.4920780385760205
EPOCH 10
training acc 0.6766005429867911, validation acc 0.6712250937068456 test acc 0.6750394573357064
validation auc 0.6343275698770849 test auc 0.6310134392753212
EPOCH 20
training acc 0.6903873959165857, validation acc 0.6829591635431052 test acc 0.6889166862554149
validation auc 0.6805825624587978 test auc 0.6820764767257763
EPOCH 30
training acc 0.6970085206691651, validation acc 0.6911422371276386 test acc 0.6972195171093724
validation auc 0.7062984983047225 test auc 0.7098176212412375
EPOCH 40
training acc 0.7033

SoftDecisionTree(
  (level1): SoftDecisionTreeLevel(
    (linear): Linear(in_features=13, out_features=1, bias=True)
  )
  (level2): SoftDecisionTreeLevel(
    (linear): Linear(in_features=13, out_features=2, bias=True)
  )
  (level3): SoftDecisionTreeLevel(
    (linear): Linear(in_features=13, out_features=4, bias=True)
  )
  (level4): SoftDecisionTreeLevel(
    (linear): Linear(in_features=13, out_features=8, bias=True)
  )
)

In [69]:
train_softtree(depth = 4, sparse=False, epochs = 150, visualize = False, verbose=True, soft_labels = True)


Soft: sparse= False, 150 epochs, 13 input dimension, 0.01 learing rate, 4 depth
EPOCH 0
training acc 0.41145647567825355, validation acc 0.4206667981850464 test acc 0.4306558312905067
validation auc 0.3891836278906889 test auc 0.3921958032580744
EPOCH 0
training acc 0.4591613107260604, validation acc 0.46865259420003946 test acc 0.4799691057456597
validation auc 0.4082335315009417 test auc 0.41144001762657506
EPOCH 10
training acc 0.6754951698020938, validation acc 0.6693549023476031 test acc 0.6751150139359952
validation auc 0.6367146206922618 test auc 0.6319229949867211
EPOCH 20
training acc 0.6899741439672793, validation acc 0.68621029788913 test acc 0.6901927532825145
validation auc 0.6739626664750514 test auc 0.6724381181698622
EPOCH 30
training acc 0.6964426989292833, validation acc 0.6925073979088577 test acc 0.6981597770240774
validation auc 0.7031948883229185 test auc 0.7036959935081881
EPOCH 40
training acc 0.7024323262043162, validation acc 0.7007220359045176 test acc 0.7068

SoftDecisionTree(
  (level1): SoftDecisionTreeLevel(
    (linear): Linear(in_features=13, out_features=1, bias=True)
  )
  (level2): SoftDecisionTreeLevel(
    (linear): Linear(in_features=13, out_features=2, bias=True)
  )
  (level3): SoftDecisionTreeLevel(
    (linear): Linear(in_features=13, out_features=4, bias=True)
  )
  (level4): SoftDecisionTreeLevel(
    (linear): Linear(in_features=13, out_features=8, bias=True)
  )
)

## Extract weight from softdecision tree, zero out weights except for one feature per node, and retrain

In [33]:
@torch.no_grad()
def extract_weights_features(softtree_model):
    #for tree models that have all features in all nodes
    weights = []
    biases = []
    features = []
    for i, (name, param) in enumerate(softtree_model.named_parameters()):    
        if i%2 == 1:
            biases.append(param.data)
        else:
            weight = torch.zeros_like(param.data)
            where = torch.argmax(torch.abs(param.data), dim=1)
            features.append(where.cpu().detach().numpy())
            weight[torch.arange(where.shape[0]), where] = param.data[torch.arange(where.shape[0]), where] 
            weights.append(weight)
    return weights, biases, features
        
full_model = train_softtree(depth = 4, sparse=False, epochs = 100, visualize = False, verbose=True)

weights, biases, features = extract_weights_features(full_model)
# print("tree with one feature in each node using pretrained weight only:")
# sparse_model = SoftDecisionTree(in_features = 13, depth=4, sparse=False, weights = weights, biases = biases)
# evaluate(sparse_model)

print("tree with one feature in each node retrained with pretrained initial weight:")
sparse_model_retrained = train_softtree(depth = 4, sparse=True, epochs = 1,  weights = weights, biases = biases, 
                                       freeze = True,  features = features, visualize = False, verbose=True, lr = 0.0001)

Soft: sparse= False, 100 epochs, 13 input dimension, 0.01 learing rate, 4 depth
before finetuning
training acc 0.5431312777164243, validation acc 0.5435786151114618 test acc 0.5652809026495181
validation auc 0.38339563289247264 test auc 0.4042055035986394
epoch 0, training loss 0.6996117234230042, validation loss 0.6923410296440125
epoch 10, training loss 0.6328803896903992, validation loss 0.6322885155677795
epoch 20, training loss 0.6010537147521973, validation loss 0.6031866073608398
epoch 30, training loss 0.5854806303977966, validation loss 0.588189959526062
epoch 40, training loss 0.5766927003860474, validation loss 0.5785672664642334
epoch 50, training loss 0.571621835231781, validation loss 0.5722051858901978
epoch 60, training loss 0.5677770972251892, validation loss 0.5672820806503296
epoch 70, training loss 0.5645943880081177, validation loss 0.5633003115653992
epoch 80, training loss 0.5621489882469177, validation loss 0.560206413269043
epoch 90, training loss 0.56019699573

## Working on visualizing the tree (wenxin)

In [635]:
@torch.no_grad()
def inverse_sigmoid(y):
    return np.log(y/(1-y))
train_x.columns

@torch.no_grad()
def visualize_disjoint_linear_layers(model, features):
    
    #visualize tree model
    #that had one non-zero weight per node
    #that trained #children linear layer per level
    weight = None
    thresholds = []
    
    feature_names = train_x.columns
    feature_names = [feature_names[feature_idx]  for level in features for feature_idx in level]
    for i, (name, param) in enumerate(model.named_parameters()):
        if i%2== 1:
            bias = torch.flatten(param.data)
            threshold = (inverse_sigmoid(0.5) + bias)/weight
            thresholds.append(threshold)
            print("{} threshold {}".format(feature_names[i//2], threshold))
        else:
            weight = torch.flatten(param.data)

# visualize_disjoint_linear_layers(sparse_model_retrained, features)


@torch.no_grad()  
def visualize_single_linear_layer(model, features=None):

    #visualize tree model
    #that had one non-zero weight per node
    #that trained one big linear layer per level
    
    feature_names = train_x.columns
#     feature_names = [feature_names[feature_idx]  for level in features for feature_idx in level]
    weight = None
    level = 0
#     print(model.level1.weight)
    for i, (name, param) in enumerate(model.named_parameters()):
#         print(name, param)
        if i%2== 1:
            bias = torch.flatten(param.data)
            threshold = (inverse_sigmoid(0.5) + bias)/weight
            for node_threshold in threshold:
                print("level {} feature {} threshold {}".format(level, feature_names[level], node_threshold))
        else:
            weight = torch.flatten(param.data)
            level+=1
    
    print(feature_names)
visualize_single_linear_layer(deterministic_sparse_model)


level 1 feature elapsed_time_u_avg threshold -1.0497232675552368
level 2 feature explanation_u_avg threshold 1.270133137702942
level 2 feature explanation_u_avg threshold -1.0496875047683716
Index(['answered_correctly_u_avg', 'elapsed_time_u_avg', 'explanation_u_avg',
       'answered_correctly_q_avg', 'elapsed_time_q_avg',
       'answered_correctly_uq_count'],
      dtype='object')
